In [14]:
import pytesseract
from PIL import Image
import numpy as np
import cv2
import unicodedata
from google.colab import files
from google.colab import output
from google.colab.patches import cv2_imshow
import fitz
import io

In [21]:
import pytesseract
from PIL import Image
import numpy as np
import cv2
import unicodedata
from google.colab import files
from IPython.display import display
import fitz
import io

def extract_text_from_pdf(pdf_file):
    pdf_reader = fitz.open(stream=pdf_file.read(), filetype="pdf")
    text = ""
    for page_num in range(pdf_reader.page_count):
        page = pdf_reader.load_page(page_num)
        text += page.get_text()
    return text

def get_text_from_image(image_path):
    # Perform OCR with pytesseract and get orientation and script information
    img = cv2.imread(image_path)
    data = pytesseract.image_to_osd(img, output_type=pytesseract.Output.DICT)
    script = data["script"]
    orientation = data["rotate"]

    # Check if the script is 'Latin' (which usually indicates typed text)
    if script != "Latin":
        return ""

    # Rotate the image to correct orientation if needed
    if orientation != 0:
        img = Image.open(image_path).rotate(360 - orientation)
        img = np.array(img)

    # Perform OCR on the rotated image
    text = pytesseract.image_to_string(Image.fromarray(img))
    return text

def is_image_present(image_path):
    img = cv2.imread(image_path)
    return img is not None

def analyze_document():
    print("Select an option:")
    print("1. Upload a PDF document")
    print("2. Scan the document using the camera")

    option = input("Enter the option number (1 or 2): ").strip()

    if option == '1':
        uploaded = files.upload()
        if len(uploaded) == 0:
            print("No file uploaded.")
            return

        file_name = list(uploaded.keys())[0]
        file_content = uploaded[file_name]

        main_text = ""
        logo_present = False
        image_present = False

        if file_name.lower().endswith('.pdf'):
            main_text = extract_text_from_pdf(io.BytesIO(file_content))
        else:
            main_text = get_text_from_image(io.BytesIO(file_content))

        if not main_text:
            print("No typed text found. Handwritten or non-Latin script detected.")
            return

        # Check if logo is present (you can add conditions based on logo characteristics)
        # For this example, I'll just check if the document is an image
        logo_present = file_name.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))

        # Check if image is present (you can add more conditions for image characteristics)
        image_present = is_image_present(file_name)

        print("Logo Present:", logo_present)
        print("Image Present:", image_present)

        uniformity_index = calculate_font_uniformity(main_text)

        print("Font Uniformity Index: {:.2f}%".format(uniformity_index))
        threshold = 20.0
        if uniformity_index < threshold:
            print("Font uniformity is low. Fonts used are not uniform. Further investigation required.")
        else:
            print("Font uniformity is acceptable. Uniform font is used. The document is original with no forgery found.")

    elif option == '2':
        capture_image_from_camera()
    else:
        print("Invalid option. Please select option 1 or 2.")

def calculate_font_uniformity(text):
    total_characters = len(text)
    unique_fonts = set()
    for char in text:
        try:
            font = unicodedata.name(char)
            if font is not None:
                unique_fonts.add(font)
        except ValueError:
            continue

    uniformity_index = (len(unique_fonts) / total_characters) * 100
    return uniformity_index

def capture_image_from_camera():
    print("Please align the document in front of the camera and press 'Capture'.")
    output.clear()
    camera = cv2.VideoCapture(0)
    _, frame = camera.read()
    camera.release()
    display(Image.fromarray(frame))
    cv2.imwrite("scanned_document.jpg", frame)
    print("Image captured successfully.")
    analyze_document()

analyze_document()


Select an option:
1. Upload a PDF document
2. Scan the document using the camera
Enter the option number (1 or 2): 1


Saving Amazon 7.pdf to Amazon 7.pdf
Logo Present: False
Image Present: False
Font Uniformity Index: 5.29%
Font uniformity is low. Fonts used are not uniform. Further investigation required.
